# Import Library #
- pip install numpy
- pip install matplotlib
- pip install pandas

In [1]:
import numpy as np
import pandas as pd
import xlrd
import pymysql




### Read Data ###

In [2]:
df = pd.read_excel('../../storage/app/public/core/DATA POTENSI CORE JULI 2024.xls', engine='xlrd', skiprows=1)
df.head()

,id,Arnet Asal,Arnet Tujuan,Ruas,Lokasi Asal,Lokasi Tujuan,Qr Code,Nama Kabel,Jenis Fo,Jenis Kabel,...,Tsel,Jumlah Titik Sambung,Join Closure,Crossing,KU to KT / KT to KU,Jembatan,Hand Hole,Main Hole,Tanda Jalur,% Hand Hole / Distance
0,1.0,ML,ML,BNR - GDI,BANTUR,GONDANG LEGI,C500389,SIEMENS,G652D,Aerial,...,0.0,93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,ML,ML,BTU - KPO,BATU,KARANG PLOSO,C500702,JEMBO,G652D,Aerial,...,0.0,144,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3.0,ML,ML,BTU - KPO,BATU,KARANG PLOSO,C500701,BICC BERCA,G655C,Aerial,...,0.0,100,5.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4.0,ML,ML,BTU - NTG,BATU,NGANTANG,C500410,MULTICO (RMJ),G655C,Aerial,...,0.0,446,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,ML,ML,BTU - NTG,BATU,NGANTANG,C500409,VOKSEL (RMJ),G655C,NaN,...,0.0,633,12.0,0.0,1.0,9.0,1.0,0.0,1.0,3000.0


### Group Total Cable ###

In [3]:
grouped_df = df['Ruas'].value_counts().reset_index()
grouped_df.columns = ['Ruas', 'Jumlah']
# print(grouped_df)

         Ruas  Jumlah
0   MLG - BLB       5
1   BLB - SGS       4
2   TUR - DPT       3
3   SGS - LWG       3
4   MLG - KEP       3
5   KEP - SBP       3
6   GDI - TUR       3
7   MLG - KLJ       3
8   KEP - GDI       2
9   MLG - SWJ       2
10  BTU - NTG       2
11  MLG - GDG       2
12  BTU - KPO       2
13  KLJ - BTU       2
14  DPT - APG       2
15  KPO - SGS       2
16  SWJ - BRG       2
17  SWJ - GDI       1
18  TMP - PKS       1
19  TWJ - SBM       1
20  NGJ - GKW       1
21  PKS - SGS       1
22  SWJ - TMP       1
23  PGK - DNO       1
24  BNR - GDI       1
25  MLG - GDI       1
26  KLJ - KPO       1
27  KEP - PGK       1
28  GDI - APG       1
29  GDG - KEP       1
30  GDG - BRG       1
31  DNO - BNR       1
32  BLB - WIT       1
33  BLB - KLJ       1
34  BLB - PKS       1
35  KLJ - GDG       1


## Counting Core ##

In [4]:
core = df.groupby('Ruas').agg({
    'Idle Baik': 'sum',
    'Idle Rusak': 'sum',
    'Core Operasi': 'sum'
})

core = core.astype(int)
# print (core)

           Idle Baik  Idle Rusak  Core Operasi
Ruas                                          
BLB - KLJ          2          40             6
BLB - PKS          9          15            24
BLB - SGS          8          16            78
BLB - WIT          0           6             0
BNR - GDI          0           4             8
BTU - KPO          9          13            14
BTU - NTG         39           7            26
DNO - BNR          0          10             2
DPT - APG          0          32            28
GDG - BRG          1           0            11
GDG - KEP         13           0            23
GDI - APG          0           2            10
GDI - TUR         16          41            45
KEP - GDI          7          27            32
KEP - PGK          2           2             8
KEP - SBP          0          42            24
KLJ - BTU          8           6            34
KLJ - GDG          0           8             4
KLJ - KPO          0           4             8
KPO - SGS    

In [5]:
pivot_table = df.pivot_table(
    index='Ruas',                # Index to group by
    aggfunc={'Idle Baik': 'sum', # Aggregation functions
             'Idle Rusak': 'sum',
             'Core Operasi': 'sum',
             'Ruas': 'count'                     
             },
)

# Convert columns to integers if necessary
pivot_table = pivot_table.astype(int)
pivot_table = pivot_table.rename(columns={'Ruas': 'Jumlah Kabel'})

pivot_table['Total'] = pivot_table[['Idle Baik', 'Idle Rusak', 'Core Operasi']].sum(axis=1)


# Display the pivot table
# print(pivot_table)

           Core Operasi  Idle Baik  Idle Rusak  Jumlah Kabel  Total
Ruas                                                               
BLB - KLJ             6          2          40             1     48
BLB - PKS            24          9          15             1     48
BLB - SGS            78          8          16             4    102
BLB - WIT             0          0           6             1      6
BNR - GDI             8          0           4             1     12
BTU - KPO            14          9          13             2     36
BTU - NTG            26         39           7             2     72
DNO - BNR             2          0          10             1     12
DPT - APG            28          0          32             2     60
GDG - BRG            11          1           0             1     12
GDG - KEP            23         13           0             1     36
GDI - APG            10          0           2             1     12
GDI - TUR            45         16          41  

## Importing To My SQL ##

In [6]:
db_host = 'localhost'
db_user = 'root'
db_password = ''
db_name = 'arnet'

connection = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)

try:
    # Create a cursor object
    cursor = connection.cursor()
    
    # Truncate table if data exists
    cursor.execute("TRUNCATE TABLE cores")
    
    # Iterate over the pivot_table DataFrame and insert each row
    for index, row in pivot_table.iterrows():
        sql = """INSERT INTO cores (segment, good, bad, used, ccount, total)
                 VALUES (%s, %s, %s, %s, %s, %s)"""
        cursor.execute(sql, (index, row['Idle Baik'], row['Idle Rusak'], row['Core Operasi'], row['Jumlah Kabel'], row['Total'] ))

    # Commit the transaction
    connection.commit()

finally:
    # Close the cursor and the connection
    cursor.close()
    connection.close()

print("Data inserted successfully")

Data inserted successfully
